In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import get_file

DATA_PATH = './data/patents-view/datasets/'
CACHE_DIR = './data/patents-view/'
DATA_TIMESTAMP='20171226'
ROOT_URL = 'http://www.patentsview.org/data/' + DATA_TIMESTAMP + '/'

In [ ]:
import sys
sys.path.append("../..")
from util.text_utils import remove_whitespace

In [ ]:
get_file('patent.tsv.zip', origin=ROOT_URL+'patent.tsv.zip', cache_dir=CACHE_DIR)
patent_feed = pd.read_csv(DATA_PATH+'patent.tsv.zip', sep='\t', error_bad_lines=False, dtype=str) #,
                        # dtype={'id': str})
patent_feed

In [ ]:
get_file('ipcr.tsv.zip', origin=ROOT_URL+'ipcr.tsv.zip', cache_dir=CACHE_DIR)
ipcr_feed = pd.read_csv(DATA_PATH+'ipcr.tsv.zip', sep='\t', error_bad_lines=False, dtype=str) # , dtype={'patent_id': str})
ipcr_feed.head()

In [ ]:
get_file('uspatentcitation.tsv.zip', origin=ROOT_URL+'uspatentcitation.tsv.zip', cache_dir=CACHE_DIR)
uspatentcitation_feed = pd.read_csv(DATA_PATH+'uspatentcitation.tsv.zip', sep='\t', error_bad_lines=False, dtype=str) # , dtype={'patent_id': str})
uspatentcitation_feed.dropna(subset=['citation_id'], inplace=True)
uspatentcitation_feed.head()

In [ ]:
set(patent_feed.id.unique())
set(ipcr_feed.patent_id.unique())
print("The intersection between patent feed and ipc feed: ", \
      len(set.intersection(set(ipcr_feed.patent_id.unique()), set(patent_feed.id.unique()))))

In [ ]:
def get_vaild_ipc_section(df):
    valid_section_symbol_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'Y']
    section = df.section
    valid_section = df.section.isin(valid_section_symbol_list)
    section[~valid_section] = np.nan
    return section 


def get_vaild_ipc_class(df):
    ipc_class_int = pd.DataFrame(pd.to_numeric(ipcr_feed.ipc_class, errors='coerce')).fillna(0.0).astype(np.int64)
    ipc_class_int[ipc_class_int > 99] = 0
    ipc_class_int = ipc_class_int.astype(str)
    ipc_class_int.ipc_class[ipc_class_int.ipc_class=='0'] = np.nan
    return ipc_class_int.ipc_class


def get_vaild_ipc_subclass(df):
    valid_subclass_symbol_list = list(map(chr, range(65, 91)))
    subclass = ipcr_feed.subclass.apply(remove_whitespace)
    valid_subclass = df.subclass.isin(valid_subclass_symbol_list)
    subclass[~valid_subclass] = np.nan
    return subclass


def clean_ipcr_feed(ipcr_feed):
    
    ipcr_feed.section = get_vaild_ipc_section(ipcr_feed)
    ipcr_feed.ipc_class = get_vaild_ipc_class(ipcr_feed)
    ipcr_feed.subclass = get_vaild_ipc_subclass(ipcr_feed)
    
    ipcr_feed['ipc4digit'] = ipcr_feed.section + ipcr_feed.ipc_class.astype(str).str.rjust(2,"0") + ipcr_feed.subclass

    print('IPCR feed: the number of invalid section symbols is ', sum(ipcr_feed.section.isnull()))
    print('IPCR feed: the number of invalid ipc_class symbols is ', sum(ipcr_feed.ipc_class.isnull()))
    print('IPCR feed: the number of invalid subclass symbols is ', sum(ipcr_feed.subclass.isnull()))
    print('IPCR feed: the number of invalid 4-digit ipc codes ', sum(ipcr_feed.ipc4digit.isnull()))
    
    return ipcr_feed

In [ ]:
ipcr_feed = clean_ipcr_feed(ipcr_feed)

### Joining patent feed and the IPC feed

In [ ]:
fused_feed = patent_feed.join(ipcr_feed[['patent_id', 'ipc4digit']].set_index('patent_id'), on='id')
fused_feed

In [ ]:
sum(~fused_feed.ipc4digit.isnull())

In [ ]:
fused_feed.date = pd.to_datetime(fused_feed.date, format='%Y-%m-%d', errors='coerce')
fused_feed['quantity'] = 1

In [ ]:
patents_count_by_kind = fused_feed[['date', 'kind', 'quantity']].groupby(['date', 'kind']).sum()
patents_count_by_kind = patents_count_by_kind.unstack(level=1).resample('M').sum()

patents_count_by_kind.plot(figsize=(20,10))

In [ ]:
top10ipc4 = list(fused_feed.groupby(['ipc4digit'])['quantity'].sum().sort_values()[-10:].index)
print("Top ipc4: ", top10ipc4)

patents_count_by_ipc4 = fused_feed[['date', 'ipc4digit', 'quantity']].groupby(['ipc4digit', 'date']).sum()
patents_count_by_ipc4 = patents_count_by_ipc4.loc[top10ipc4]
patents_count_by_ipc4 = patents_count_by_ipc4.unstack(level=0).resample('M').sum()

patents_count_by_ipc4.plot(figsize=(20,10))

In [ ]:
patents_count_tot = fused_feed[['date', 'quantity']].groupby(['date']).sum()
patents_count_tot = patents_count_tot.resample('M').sum()

patents_count_tot.plot(figsize=(20,10))

### Distribution over IPC categories

In [ ]:
patents_by_ipc4 = fused_feed.reset_index(drop=True)[['ipc4digit', 'quantity']].groupby('ipc4digit').sum()
patents_by_ipc4.sort_values(by='quantity', ascending=False, inplace=True)

ax1 = patents_by_ipc4.plot(kind='line', use_index=False, logy=True, figsize=(20, 5))
ax1.set_title("Distribution of the total patent quantity by IPC4")
ax1.set_xlabel(r'IPC4 code ordered by quantity')
ax1.set_ylabel('Total Quantity, log scale')

plt.show()

In [ ]:
# sum(patents_by_ipc4[:1000].astype(np.int64)) / sum(patents_by_ipc4.astype(np.int64))
num_categories_cutoff = 1000
print(num_categories_cutoff, "out of", len(patents_by_ipc4),
      " ipc4 categories account for %2.2f%% patents" 
      %(100*sum(patents_by_ipc4[:1000].quantity) / sum(patents_by_ipc4.quantity)))

### Citation analysis

In [ ]:
# fused_feed.date = pd.to_datetime(fused_feed.date, format='%Y-%m-%d', errors='coerce')
def get_vaild_citation_id(df):
    # uppercase, remove all but numbers and AIXHTREPD
    citation_id = df.citation_id.str.upper().replace("[^AIXHTREPD0-9]", value='', regex=True) # .apply(remove_whitespace)
    # remove leading zeros
    citation_id = citation_id.replace("^0+", value='', regex=True)
    return citation_id


uspatentcitation_feed.citation_id = get_vaild_citation_id(uspatentcitation_feed)

print("Number of unique cited patents", len(uspatentcitation_feed.citation_id.unique()))
print("Number of unique citing patents", len(uspatentcitation_feed.patent_id.unique()))

In [ ]:
uspatentcitation_feed["citation_count"] = 1
uspatentcitation_bypatent = uspatentcitation_feed[['citation_id', 'citation_count']].groupby('citation_id').count() \
    .sort_values(by='citation_count', ascending=False)

In [ ]:
ax1 = uspatentcitation_bypatent.plot(kind='line', use_index=False, logy=True, figsize=(20, 5))
ax1.set_title("Distribution of the number of citations by patent")
ax1.set_xlabel(r'Patent ordered by the number of citations')
ax1.set_ylabel('Number of citations, log scale')

plt.show()

In [ ]:
uspatentcitation_bypatent

In [ ]:
uspatentcitation_bypatent[:10]
# Request that can be used to obtain the number of citations for a given patent
# http://www.patentsview.org/api/patents/query?q={%22_and%22:[{%22patent_number%22:%227674650%22}]}&f=[%22patent_num_cited_by_us_patents%22]

In [ ]:
fused_feed = fused_feed.join(uspatentcitation_bypatent, on='id')

In [ ]:
fused_feed[fused_feed.citation_count.isnull()]

In [ ]:
uspatentcitation_bypatent.loc['RE46649']

In [ ]:
sum(uspatentcitation_feed['citation_id'].str.contains(".*4683202.*", regex=True))

In [ ]:
sum(uspatentcitation_feed['patent_id'].str.contains(".*4683202.*", regex=True))

In [ ]:
len(uspatentcitation_feed['citation_id'].unique())

In [ ]:
sum(citation_id.str.contains(".*7674650.*", regex=True))

In [ ]:
sum(citation_id.str.contains("7674650", regex=False))

In [ ]:
brf_sum_text_feed = pd.read_csv(DATA_PATH+'brf_sum_text.zip', sep='\t', encoding="ISO-8859-1", nrows=10)
brf_sum_text_feed

In [ ]:
brf_sum_text_feed['text'][2]

In [ ]:
claim_feed = pd.read_csv(DATA_PATH+'claim.zip', sep='\t', encoding="ISO-8859-1", nrows=10)
claim_feed

In [ ]:
claim_feed['text'][0]